# Adquisición de convocatorias por APIs #

Este notebook consiste en conectarse a las APIs de las bases de datos donde se encuentran las convocatorias españolas y europeas y adquirir los datos.

## 1. Convocatorias españolas ##

In [46]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.


In [47]:
# Importar librerías
import requests
import pandas as pd
import datetime
import time
import openpyxl

### 1.1 Lista de convocatorias ###

La primera llamada a la API se hace para obtener la  lista de todas las convocatorias. Estos son los 
parámtros que se pueden usar:

In [ ]:
posibles_params = {
    "page": 0,  # Número de página (empieza en 0)
    "pageSize": 50,  # Tamaño de página
    "order": "numeroConvocatoria",  # Campo por el que ordenar
    "direccion": "asc",  # Sentido de la ordenación: 'asc' o 'desc'
    "vpd": "GE",  # Identificador del portal
    "descripcion": "Resolución",  # Texto a buscar en el título o descripción
    "descripcionTipoBusqueda": 0,  # 0: frase exacta, 1: todas las palabras, 2: alguna palabra
    "numeroConvocatoria": "376046",  # Código BDNS a buscar
    "mrr": False,  # Mecanismo de recuperación y resiliencia
    "fechaDesde": "18/12/2017",  # Fecha de inicio (dd/mm/yyyy)
    "fechaHasta": "18/12/2017",  # Fecha de fin (dd/mm/yyyy)
    "tipoAdministracion": "C",  # 'C', 'A', 'L', 'O'
    "organos": ["713", "4730"],  # Lista de identificadores de órganos administrativos
    "regiones": [3, 50],  # Lista de identificadores de regiones
    "tiposBeneficiario": [3],  # Lista de identificadores de tipos de beneficiarios
    "instrumentos": [1],  # Lista de identificadores de instrumentos de ayuda
    "finalidad": 11,  # Identificador de la finalidad de la política de gasto
    "ayudaEstado": "SA.45221"  # Código de ayuda de estado
}


In [41]:
# Configuración de parámetros
base_url = "https://www.pap.hacienda.gob.es/bdnstrans/api/convocatorias/busqueda"
vpd = "GE"  # Identificador del portal, según la docu
page_size = 25
max_paginas = 3  # Puedes aumentar si quieres más resultados

resultados = []


# 2. Probar la API con parámetros y cabecera Accept: application/json
params = {
    "vpd": vpd,
    "page": 0,
    "pageSize": page_size
}
headers = {"Accept": "application/json"}


# 3. Si la respuesta es JSON, continuar con la descarga paginada
if "application/json" in r2.headers.get("Content-Type", ""):
    print("✅ La API responde con JSON. Descargando datos paginados...")
    for pagina in range(0, max_paginas):
        print(f"📄 Cargando página {pagina}...")
        params["page"] = pagina
        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            convocatorias = data.get("convocatorias", data.get("content", []))  # content es común en APIs paginadas
            if not convocatorias:
                print("✅ No hay más datos.")
                break
            resultados.extend(convocatorias)
            time.sleep(0.5)  # para evitar sobrecargar la API
        except Exception as e:
            print(f"❌ Error en la página {pagina}: {e}")
            break
    # Convertir a DataFrame y mostrar
    df = pd.DataFrame(resultados)
    print("Columnas disponibles:", df.columns.tolist())
    # Mostrar las primeras columnas si existen
    cols = [c for c in ["id", "titulo", "organoConvocante", "fechaPublicacion"] if c in df.columns]
else:
    print("❌ La API no responde con JSON. Revisa los parámetros, la URL o si la API está disponible.")

✅ La API responde con JSON. Descargando datos paginados...
📄 Cargando página 0...
📄 Cargando página 1...
📄 Cargando página 2...
Columnas disponibles: ['id', 'mrr', 'numeroConvocatoria', 'descripcion', 'descripcionLeng', 'fechaRecepcion', 'nivel1', 'nivel2', 'nivel3', 'codigoInvente']


In [48]:
df.head(10)  # Mostrar las primeras filas del DataFrame

# Guardar en un archivo Excel
output_file = "listado_convocatorias.xlsx"
df.to_excel(output_file, index=False)

### 1.2 Búsqueda de una convocatoria en específico ###

Si queremos buscar una convocatoria específica lo haremos a partir del id obtenido en la primera búsqueda:

In [ ]:

base_url = "https://www.infosubvenciones.es/bdnstrans/api/convocatorias"
params = {
    "vpd": "GE",         # Cambia por el portal que te interese
    "numConv": "842695"   # Número de convocatoria
}
headers = {"Accept": "application/json"}

print("🔎 Consultando convocatoria por parámetros...")
r = requests.get(base_url, params=params, headers=headers)
print("Status code:", r.status_code)
print("URL final:", r.url)
print("Content-Type:", r.headers.get("Content-Type"))
print("="*60)

if "application/json" in r.headers.get("Content-Type", ""):
    data = r.json()
    # Si la respuesta es una lista, conviértela directamente
    if isinstance(data, list):
        convocatoria = pd.DataFrame(data)
    # Si es un dict, conviértelo en DataFrame de una fila
    elif isinstance(data, dict):
        convocatoria = pd.DataFrame([data])
    else:
        print("Respuesta inesperada:", data)
        convocatoria = pd.DataFrame()
    print("Columnas disponibles:",convocatoria.columns.tolist())
else:
    print("❌ La API no responde con JSON. Revisa los parámetros, la URL o si la API está disponible.")
    convocatoria = pd.DataFrame()

🔎 Consultando convocatoria por parámetros...
Status code: 200
URL final: https://www.infosubvenciones.es/bdnstrans/api/convocatorias?vpd=GE&numConv=842695
Content-Type: application/json
Columnas disponibles: ['id', 'organo', 'sedeElectronica', 'codigoBDNS', 'fechaRecepcion', 'instrumentos', 'tipoConvocatoria', 'presupuestoTotal', 'mrr', 'descripcion', 'descripcionLeng', 'tiposBeneficiarios', 'sectores', 'regiones', 'descripcionFinalidad', 'descripcionBasesReguladoras', 'urlBasesReguladoras', 'sePublicaDiarioOficial', 'abierto', 'fechaInicioSolicitud', 'fechaFinSolicitud', 'textInicio', 'textFin', 'ayudaEstado', 'urlAyudaEstado', 'fondos', 'reglamento', 'objetivos', 'sectoresProductos', 'documentos', 'anuncios', 'advertencia']


In [49]:
# Ejemplo de respuesta de una convocatoria
convocatoria

# Guardar la convocatoria en un archivo Excel
convocatoria_file = "convocatoria_842695.xlsx"
convocatoria.to_excel(convocatoria_file, index=False)

### 1.3 Adquisición de documentos ###

Finalmente, para acceder a un documento, se utiliza la llamada a la API específica a documentos a partir del id obtenido en la llamada anterior:

In [ ]:
# Supón que ya tienes el DataFrame 'convocatoria' y quieres descargar todos los documentos
docs = convocatoria.iloc[0]['documentos']  # Si solo hay una convocatoria

# Carpeta donde guardar los documentos
os.makedirs("documentos_convocatoria", exist_ok=True)

for doc in docs:
    id_doc = doc['id']
    nombre = doc.get('nombreFic', f"documento_{id_doc}.pdf")
    url = f"https://www.infosubvenciones.es/bdnstrans/api/convocatorias/documentos?idDocumento={id_doc}"
    print(f"Descargando {nombre} ...")
    resp = requests.get(url)
    if resp.status_code == 200:
        with open(os.path.join("documentos_convocatoria", nombre), "wb") as f:
            f.write(resp.content)
        print(f"✅ Guardado: {nombre}")
    else:
        print(f"❌ Error al descargar {nombre} (status {resp.status_code})")

Descargando TextoConvocatoria.pdf ...
✅ Guardado: TextoConvocatoria.pdf


# 2. Lectura de documentos con NLP #